In [2]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import time
import os
import glob

In [3]:
%run py/constants.py
%run py/take_processing_methods.py
%run py/marker_processing_methods.py
%run py/heatmap_methods.py
%run py/grasp_property_methods.py
%run py/rasterization_methods.py

ERROR:root:File `'py/labeling_names.py'` not found.


In [5]:
def read_take(take_file):
    """
    Reads the given take file and returns it in pandas format. 
    RETURN: Pandas dataframe, recording start as datetime
    """
    f_csv = open(take_file, 'r')
    marker_names = []   
    coordinates = [] 
    marker_type = []
    time_string = ""

    for i in range(0, 7):
        line = f_csv.readline()

        if i == 0:
            time_string = line.split(',')[9]
        if i == 3:
            marker_names = line.split(",")
        if i == 5:
            marker_type = line.split(",")
        if i == 6:
            coordinates = line.split(",")
    
    # Create the list of unique marker names to pass to the read_csv function.
    for i in range(0, len(marker_names)):
        newname = None
        for jname in joints:
            if jname.lower() in marker_names[i].lower() and not ("fka" in marker_names[i].lower()):
                newname = jname + "_" + str(coordinates[i])
                marker_names[i] = newname.strip()
        
        if newname == None:
            newname = marker_names[i] + "_" + str(coordinates[i])
            marker_names[i] = newname.strip()
            
        if marker_type[i].strip() != "Position":
            marker_names[i] = marker_names[i] + "_" + marker_type[i]
                
    marker_names[0] = 'Frame'
    marker_names[1] = 'Time'
    f_csv.close()
    
    # Importing one take (will be all takes later on)
    df = pd.read_csv(take_file, skiprows=list(range(0,7)), names=marker_names)
    df = df.set_index(df.Frame, drop=False)
    # Drop all irrelevant information
    
    
    # retrieve datetime and fix 12AM error of motive
    dt = datetime.strptime(time_string, '%Y-%m-%d %I.%M.%S.%f %p')
    if dt.hour == 0 and dt.strftime("%p") == "AM":
        dt = dt.replace(hour=12)
    
    return df, dt

def quaternion_matrix(quat):
    """
    Converts a quaternion to a rotation matrix.
    Source: http://fabiensanglard.net/doom3_documentation/37726-293748.pdf
    RETURN: Rotation matrix as numpy array.
    """
    x = quat[0]
    y = quat[1]
    z = quat[2]
    w = quat[3]

    return np.array([
        [1.0-2*y*y-2*z*z, 2.0*x*y+2*w*z, 2.0*x*z - 2*w*y],
        [2.0*x*y - 2*w*z, 1.0-2*x*x-2*z*z, 2.0*y*z+2*w*x],
        [2.0*x*z+2*w*y, 2.0*y*z-2*w*x, 1.0-2*x*x-2*y*y]])

def get_transformed_marker(frame_index, marker_id, df, rot_matrices):
    rot_matrix = rot_matrices[df.index.get_loc(frame_index)].T

    rb_pivot =[df[df.columns[6]].ix[frame_index], df[df.columns[7]].ix[frame_index], df[df.columns[8]].ix[frame_index]]

    marker = [df[marker_id + "_X"].ix[frame_index], df[marker_id + "_Y"].ix[frame_index], df[marker_id + "_Z"].ix[frame_index]]  
    marker = inv_trans(marker, rb_pivot, rot_matrix)

    return marker

def quaternion_matrix(row):
    """
    Converts a quaternion to a rotation matrix.
    Source: http://fabiensanglard.net/doom3_documentation/37726-293748.pdf
    RETURN: Rotation matrix as numpy array.
    """
    x = row[rigid_body_name + '_X_Rotation']
    y = row[rigid_body_name + '_Y_Rotation']
    z = row[rigid_body_name + '_Z_Rotation']
    w = row[rigid_body_name + '_W_Rotation']
    
    rot_matrix = [
        1.0-2*y*y-2*z*z, 2.0*x*y+2*w*z, 2.0*x*z - 2*w*y,
        2.0*x*y - 2*w*z, 1.0-2*x*x-2*z*z, 2.0*y*z+2*w*x,
        2.0*x*z+2*w*y, 2.0*y*z-2*w*x, 1.0-2*x*x-2*y*y]
    
    return rot_matrix


def inv_trans(p, pos, rot):
    """
    Transform a marker into the rigid body coordinate system
    """
    p = np.subtract(p, pos)
    p = np.dot(p, rot.T)
    p = [p[0], p[2], p[1]]
    return p

def trans(prow):
    markers = ["Wrist", "R_Shape_1","R_Shape_2","R_Shape_3","R_Shape_4",
               "Thumb_Fn", "Thumb_DIP", "Thumb_PIP" ,"Thumb_MCP", "Index_Fn", "Index_DIP", "Index_PIP","Index_MCP", 
               "Middle_Fn", "Middle_DIP", "Middle_PIP", "Middle_MCP", "Ring_Fn", "Ring_DIP", "Ring_PIP","Ring_MCP", 
               "Little_Fn", "Little_DIP", "Little_PIP", "Little_MCP"]
    row = prow.copy(deep=True)
    
    # iterate over all rows
    for m in markers:
        
        x = row[m + "_X"]
        y = row[m + "_Y"]
        z = row[m + "_Z"]
        
        pivot = row['Pivot']
        rot_matrix = row['RotationMatrix']
        
        if not math.isnan(x):
            marker = [x, y, z]
            marker = inv_trans(marker, pivot, rot_matrix)
            
            row[m + "_X"] = marker[0]
            row[m + "_Y"] = marker[1]
            row[m + "_Z"] = marker[2]
            
    return row

def log(s):
    with open("./out/status_PY01_preprocessing.txt", "a") as myfile:
        myfile.write("[" + str(datetime.now()) + "] " + s + "\n")
    print("[" + str(datetime.now()) + "] " + s)


test_participants = [1,2,9,15,19]

for filename in os.listdir("raw-data/"):
    if filename.endswith(".csv"):
        current_participant = filename.split(".")[0]
        cp = int(current_participant.replace("P","").split("_")[0])
        if cp not in test_participants:
            rigid_body_name = filename.split("_")[1] 
            #RIGID_BODY_NAMES[current_participant]

            #if os.path.isfile("./data/transformed_optitrack/dfTransformed_" + str(current_participant) + ".pkl") :
            #    log(current_participant + " is already available. Skipped.")
            #    continue

            log("Reading " + current_participant)
            df, dt = read_take("raw-data/" + filename)
            TEST = 0
            df['Time'] = ((df['Time'] + dt.timestamp()) * 1000) + TEST # this is in ms

            # Cells for quaternions
            x = df[df.columns[2]]
            y = df[df.columns[3]]
            z = df[df.columns[4]]
            w = df[df.columns[5]]

            rot_matrix = np.array([
                [1-2*y*y-2*z*z, 2*x*y+2*w*z, 2*x*z - 2*w*y],
                [2*x*y - 2*w*z, 1-2*x*x-2*z*z, 2*y*z+2*w*x],
                [2*x*z+2*w*y, 2*y*z-2*w*x, 1-2*x*x-2*y*y]])

            angle = np.degrees(np.arccos(np.dot((rot_matrix[:,1,:].T), [0, 1, 0])))
            #adf = pd.DataFrame(angle)
            #df2 = adf[np.logical_not(adf[0].isnull())]
            #df3 = df2[df2[0] < df2[0].mean() + df2[0].std()*2]
            dfFinal = df#.iloc[df3.index]

            log("Creating RotationMatrix")
            dfFinal['RotationMatrix'] = dfFinal.apply(lambda row : quaternion_matrix(row), axis=1)
            dfFinal.RotationMatrix = dfFinal.RotationMatrix.apply(lambda x : np.array(x).reshape(3,3))
            dfFinal['Pivot'] = dfFinal.apply(lambda row : [row[rigid_body_name + '_X'], row[rigid_body_name + '_Y'], row[rigid_body_name + '_Z']], axis=1)
            dfFinal['Pivot'] = dfFinal.Pivot.apply(lambda x : np.array(x))

            log("Applying transformation")
            dfTransformed = dfFinal.apply(lambda x : trans(x), axis=1)
            log("Saving to File")
            dfTransformed.to_pickle("Pickles/dfTransformed_" + current_participant + ".pkl")

[2020-02-14 14:01:52.013105] Reading P5_OPO_seated
[2020-02-14 14:01:53.972573] Creating RotationMatrix
[2020-02-14 14:02:04.435048] Applying transformation
[2020-02-14 14:07:01.624820] Saving to File
[2020-02-14 14:07:02.535420] Reading P4_OPO_walking
[2020-02-14 14:07:04.707808] Creating RotationMatrix
[2020-02-14 14:07:17.720396] Applying transformation
[2020-02-14 14:13:17.253124] Saving to File
[2020-02-14 14:13:18.311778] Reading P5_N6_walking
[2020-02-14 14:13:20.592766] Creating RotationMatrix
[2020-02-14 14:13:33.912038] Applying transformation
[2020-02-14 14:20:52.417436] Saving to File
[2020-02-14 14:20:53.767994] Reading P11_S4_walking
[2020-02-14 14:20:56.250098] Creating RotationMatrix
[2020-02-14 14:21:13.352008] Applying transformation
[2020-02-14 14:28:54.828814] Saving to File
[2020-02-14 14:28:56.316159] Reading P16_S4_walking
[2020-02-14 14:28:58.816294] Creating RotationMatrix
[2020-02-14 14:29:14.495976] Applying transformation
[2020-02-14 14:35:54.063292] Saving 

[2020-02-14 19:22:36.674576] Saving to File
[2020-02-14 19:22:37.648727] Reading P8_S4_seated
[2020-02-14 19:22:39.128195] Creating RotationMatrix
[2020-02-14 19:22:48.321143] Applying transformation
[2020-02-14 19:27:28.478935] Saving to File
[2020-02-14 19:27:29.404861] Reading P19_N6_seated
[2020-02-14 19:27:31.224874] Creating RotationMatrix
[2020-02-14 19:27:44.549317] Applying transformation
[2020-02-14 19:33:28.062900] Saving to File
[2020-02-14 19:33:29.198663] Reading P3_S3_walking
[2020-02-14 19:33:31.674395] Creating RotationMatrix
[2020-02-14 19:33:47.659491] Applying transformation
[2020-02-14 19:41:37.078096] Saving to File
[2020-02-14 19:41:38.720187] Reading P16_S4_seated
[2020-02-14 19:41:40.904624] Creating RotationMatrix
[2020-02-14 19:41:54.207765] Applying transformation
[2020-02-14 19:47:33.295918] Saving to File
[2020-02-14 19:47:34.507059] Reading P17_OPO_seated
[2020-02-14 19:47:36.172785] Creating RotationMatrix
[2020-02-14 19:47:46.323713] Applying transforma

[2020-02-14 23:35:13.080898] Applying transformation
[2020-02-14 23:43:44.136875] Saving to File
[2020-02-14 23:43:45.829408] Reading P4_N6_walking
[2020-02-14 23:43:48.410262] Creating RotationMatrix
[2020-02-14 23:44:03.911930] Applying transformation
[2020-02-14 23:51:32.678664] Saving to File
[2020-02-14 23:51:34.085230] Reading P17_N6_seated
[2020-02-14 23:51:36.025788] Creating RotationMatrix
[2020-02-14 23:51:47.957179] Applying transformation
[2020-02-14 23:57:36.723415] Saving to File
[2020-02-14 23:57:37.917458] Reading P10_N6_sitting
[2020-02-14 23:57:40.712401] Creating RotationMatrix
[2020-02-14 23:57:57.408808] Applying transformation
[2020-02-15 00:06:04.407416] Saving to File
[2020-02-15 00:06:06.004868] Reading P21_S4_walking
[2020-02-15 00:06:07.890486] Creating RotationMatrix
[2020-02-15 00:06:20.755216] Applying transformation
[2020-02-15 00:12:02.134155] Saving to File
[2020-02-15 00:12:03.266459] Reading P4_S4_walking
[2020-02-15 00:12:05.134562] Creating Rotation

[2020-02-15 04:03:01.147159] Creating RotationMatrix
[2020-02-15 04:03:14.095535] Applying transformation
[2020-02-15 04:09:44.620791] Saving to File
[2020-02-15 04:09:45.905232] Reading P7_OPO_walking
[2020-02-15 04:09:48.002072] Creating RotationMatrix
[2020-02-15 04:10:01.709302] Applying transformation
[2020-02-15 04:15:53.286824] Saving to File
[2020-02-15 04:15:54.462673] Reading P3_N6_seated
[2020-02-15 04:15:56.874010] Creating RotationMatrix
[2020-02-15 04:16:10.858693] Applying transformation
[2020-02-15 04:22:54.508772] Saving to File
[2020-02-15 04:22:55.947623] Reading P19_S3_seated
[2020-02-15 04:22:57.793038] Creating RotationMatrix
[2020-02-15 04:23:11.767632] Applying transformation
[2020-02-15 04:29:19.260612] Saving to File
[2020-02-15 04:29:20.414624] Reading P3_N6_walking
[2020-02-15 04:29:23.147153] Creating RotationMatrix
[2020-02-15 04:29:41.337231] Applying transformation
[2020-02-15 04:38:03.011481] Saving to File
[2020-02-15 04:38:04.723803] Reading P13_S4_se